# M1 Text Processing Using spaCy Library

## Objective

Preprocess the dataset using spaCy library.

## Load all relevant Python libraries and a spaCy language model.

In [209]:
import json
import spacy

In [86]:
# !python -m spacy download en_core_web_sm

In [210]:
sp = spacy.load("en_core_web_sm")
stopwords = sp.Defaults.stop_words

##  Open the provided JSON file. 

It contains a list of dictionaries with summaries from Wikipedia articles, where each dictionary has three key-value pairs. The keys title, text and url correspond to:

- Title of the Wikipedia article the text is taken from.


- Wikipedia article text. (In this dataset we included only the summary.)


- Link to the Wikipedia article.

In [325]:
with open('data/data.json', 'r') as outfile:
    summaries = json.load(outfile)
print(summaries[0].keys())

dict_keys(['title', 'text', 'url'])


In [326]:
summaries[0]['text']

'A pandemic (from Greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. A widespread endemic disease with a stable number of infected people is not a pandemic. Widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.\nThroughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. The most fatal pandemic in recorded history was the Black Death (also known as The Plague), which killed an estimated 75–200 million people in the 14th century. The term was not used yet but was for later pandemics including the 1918 influenza pandemic (Spanish flu). Current pandemics include COVID-19 (SARS-CoV-2) and HIV/AIDS.'

In [327]:
len(summaries)

26

## Create a Python function that takes in a text string, performs all operations described in the previous step, and outputs a list of tokens (lemmas).

- Lowercases the text string.


- Creates a spaCy document with the text lemmas and their attributes using a spaCy model of your choice.


- Removes stop words, punctuation, and other unclassified lemmas.


- Returns a list of tokens (lemmas) found in the text.

In [213]:
# Lowercase data. Lowercase the text
# Explore the attributes of each token returned SpaCy.
text = summaries[0]['text']
text_tokenized = sp(text.lower())
for token in text_tokenized[:5]:
    print(type(token), token.text, token.pos_, token.dep_, token.lemma_)

<class 'spacy.tokens.token.Token'> a DET det a
<class 'spacy.tokens.token.Token'> pandemic ADJ nsubj pandemic
<class 'spacy.tokens.token.Token'> ( PUNCT punct (
<class 'spacy.tokens.token.Token'> from ADP prep from
<class 'spacy.tokens.token.Token'> greek ADJ amod greek


In [214]:
text_tokenized

a pandemic (from greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. a widespread endemic disease with a stable number of infected people is not a pandemic. widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.
throughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. the most fatal pandemic in recorded history was the black death (also known as the plague), which killed an estimated 75–200 million people in the 14th century. the term was not used yet but was for later pandemics including the 1918 influenza pandemic (spanish flu). current pandemics include covid-19 (sars-cov-2) and hiv/aids.

In [215]:
def lower(text):
    return sp(text.lower())

In [216]:
lower(summaries[0]['text'])

a pandemic (from greek πᾶν, pan, "all" and δῆμος, demos, "people") is an epidemic of an infectious disease that has spread across a large region, for instance multiple continents or worldwide, affecting a substantial number of people. a widespread endemic disease with a stable number of infected people is not a pandemic. widespread endemic diseases with a stable number of infected people such as recurrences of seasonal influenza are generally excluded as they occur simultaneously in large regions of the globe rather than being spread worldwide.
throughout human history, there have been a number of pandemics of diseases such as smallpox and tuberculosis. the most fatal pandemic in recorded history was the black death (also known as the plague), which killed an estimated 75–200 million people in the 14th century. the term was not used yet but was for later pandemics including the 1918 influenza pandemic (spanish flu). current pandemics include covid-19 (sars-cov-2) and hiv/aids.

In [217]:
# Remove stop words and punctuation
def remove_redundant_tokens(text_tokenized):
    return ' '.join([token.text for token in text_tokenized if not token.is_stop and not token.is_punct])

In [218]:
# Lemmatize (tokenize) the texts
def lemmatize(text):
    return [token.lemma_ for token in sp(text)]

In [219]:
# Build a tokenizer function
def tokenizer(document):
    """
    This function accepts a text string and:
    1. Lowercases it
    2. Removes redundant tokens
    3. Performs token lemmatization
    """ 
    text_tokenized = lower(document)
    clean_text =  remove_redundant_tokens(text_tokenized)
    token_lemmatized = lemmatize(clean_text)
    return token_lemmatized

## Use this function to preprocess all text documents in the dataset (text field only), and add the resulting lists to the dictionaries from step 1. 

You should end up with a list of dictionaries, each of which now has four key-value pairs:

- title: Title of the Wikipedia article the text is taken from.


- text: Wikipedia article text. (In this dataset we included only the summary.)


- tokenized_text: Tokenized Wikipedia article text.


- url: Link to the Wikipedia article.

In [220]:
# Preprocess all the documents using the tokenizer function
for doc in summaries:
    doc['tokenized_text'] = tokenizer(doc['text'])

In [221]:
summaries[0]['tokenized_text'][:10]

['pandemic',
 'greek',
 'πᾶν',
 'pan',
 'δῆμος',
 'demos',
 'people',
 'epidemic',
 'infectious',
 'disease']

In [222]:
len(summaries[0]['tokenized_text'])

88

## Save the new list of dictionaries in JSON format.

In [223]:
# Save the tokenized texts to file:
with open('data/summaries.json', 'w') as outfile:
    json.dump(summaries, outfile)
outfile

<_io.TextIOWrapper name='data/summaries.json' mode='w' encoding='UTF-8'>

# M2 TF-IDF Search Using Cosine Similarity

## Objective

Implement a basic Tf-Idf search.

- In your search for an optimal document retrieval method in the CDC’s huge knowledge base, you decide to try the term frequency search first because of its simplicity. It is a well-developed technique and is a great place to start!


- In Milestone 1, you prepared the documents for Tf-Idf-based search. You also computed the Tf-Idf vectors for every document in the CDC’s knowledge base. The standard approach to finding the most relevant documents to your query is to compute similarities between the Tf-Idf vectors of the documents and the query. It works, but you realize that it can be very inefficient for very large document sets since you need to compute the similarities between the query and every one of the documents in your database. What would be a better solution? Let us move on to the last milestone of the project to find out!

## Load all relevant Python libraries and a spaCy language model.

In [206]:
import json
import itertools
from collections import Counter

import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

## Access the tokenized text in your new dataset from the previous milestone. 

Each document dictionary should now include a new key-value pair with the lemmatized text of the articles.

In [328]:
with open('data/summaries.json', 'r') as f:
    data = json.load(f)
f.close()

## Create a corpus vocabulary. It should simply be a list of unique tokens in the provided set of documents. 

Count how many times each unique token appears in the corpus, you will need these counts for the next step.

In [395]:
# concatenate all tokenized texts into a single list
tokenized_texts = [i["tokenized_text"] for i in data]
print(np.array(tokenized_texts).shape)

# flatten the list of lists (use itertools.chain)
flattened_tokenized_texts = list(itertools.chain(*tokenized_texts))
print(np.array(flattened_tokenized_texts).shape)

# remove duplicates
vocab = list(set(flattened_tokenized_texts))
len(vocab)

(26,)
(3617,)


1506

In [396]:
# Save the vocabulary as a json file
with open('data/vocab.json', 'w') as outfile:
    json.dump(vocab, outfile)
outfile.close()

In [275]:
# count how many times each token occurs in a document
# you will need it for TfIdf calculations
docs_token_counter = []
for doc in data:
    # For each document, count how many of each token they have
    # Counter function from collections is very handy
    docs_token_counter.append([y for x, y in Counter(doc['tokenized_text']).items()])

In [322]:
len(docs_token_counter)

26

In [294]:
number_docs_with_token  = {}
for token in vocab:
   # For each token in corpus vocabulary, count in how many documents it occurs
    doc_count = 0
    for document in data:
        if token in document['tokenized_text']:
            doc_count = doc_count + 1
    number_docs_with_token[token] = doc_count

In [296]:
[v for v in number_docs_with_token.values() if v > 1][:10]

[4, 2, 16, 2, 3, 4, 3, 2, 2, 2]

In [401]:
number_docs_with_token['ebola']

2

## Calculate Tf-Idf vectors for every article in the dataset and add these vectors to the article dictionaries. 

You should end up the same list of dictionaries as before, but with a new key-value pair containing Tf-Idf vectors:

- title: Title of the Wikipedia article the text is taken from.


- text: Wikipedia article text. (In this dataset we included only the summary.)


- tokenized_text: Tokenized Wikipedia article text.


- url: Link to the Wikipedia article.


- tf_idfs: Tf_Idf vector.

$tf = \frac{count(token\:in\:document)}{count(all\tokens\:in\:document)}$


$idf(token) = \frac{number\:of\:documents}{number\:of\:documents\:containing\:the\:token}$

In [462]:
for i, doc in enumerate(docs_token_counter):
    doc_length = len(doc)
    tfidf_vec = []
    for token in vocab:
        # compute a term frequency (tf) per document
        tf = doc[token] / len(data[i]["tokenized_text"])

        # compute a log of inverse document frequency per document
        idf = np.log(len(data)/number_docs_with_token[token])
        
        # Compute tfidf for the token and append to a list of TfIdfs for this document
        tfidf_vec.append(tf*idf)
    
    # add tf_idf vector to the corresponding data dictionary
    data[i]['tfidf_vec'] = tfidf_vec       

In [463]:
# Save an updates summary with computed Tf-Idf vectors
with open('data/summaries.json', 'w') as json_file:
    json.dump(data, json_file)
json_file.close()

In [404]:
query = "highest pandemic casualties"

In [445]:
# Reuse the workflow for article Tf-Idf calculation
# to build a vectorizer function for search queries

def vectorize(query, vocab = vocab): 
    query_vec = []
    # Tokenize query
    tokenized_query = tokenizer(query)
    query_length = len(tokenized_query)
    # Count unique tokens in query
    for token in vocab:
        # Build a TfIdf vector of the same shape as the document TfIdfs
        tf = Counter(tokenized_query)[token]/query_length
        idf = np.log(len(data)/number_docs_with_token[token])
        query_vec.append(tf*idf)
        
    return query_vec        

## Now we can try to search our list of dictionaries using this Tf-Idf field using existing tools for similarity. 

We suggest you use scikit-learn library and its cosine_similarity function.

In [454]:
vec_1 = np.array(vectorize(query)).reshape(1, -1)

In [467]:
vec_2 = np.array(data[0]['tfidf_vec']).reshape(1, -1)

In [468]:
cosine_similarity(vec_1, vec_2)[0][0]

0.0160686743357183

In [460]:
# Build a search function
def search_tfidf(query, docs):    
    # vectorize query
    vectorized_query = vectorize(query)
    # Build a list of results using sklearn cosine_similarity function
    rankings = []
    for doc in docs:
        # compute cosine similarity rank
        rank = cosine_similarity(np.array(vectorized_query).reshape(1, -1), np.array(doc['tfidf_vec']).reshape(1, -1))[0][0]
        if rank > 0:
            # add this document to results along with its similarity rank
            rankings.append({'title': doc['title'], 'rank': rank})
     
    # The results should be a list of dictionaries with at least the 'title' and 'rank' fields
    return sorted(rankings, key=lambda item: item.get("rank"), reverse=True)

In [461]:
# Lets test how well this fuction works
search_tfidf("ebola", data)

[{'title': 'Plague of Cyprian', 'rank': 0.11778345241757451},
 {'title': 'Science diplomacy and pandemics', 'rank': 0.07118947137494436}]

In [440]:
for s in data:
    if s["title"] == 'Plague of Cyprian':
        print(s["text"])

The Plague of Cyprian was a pandemic that afflicted the Roman Empire about from AD 249 to 262. The plague is thought to have caused widespread manpower shortages for food production and the Roman army, severely weakening the empire during the Crisis of the Third Century. Its modern name commemorates St. Cyprian, bishop of Carthage, an early Christian writer who witnessed and described the plague. The agent of the plague is highly speculative because of sparse sourcing, but suspects have included smallpox, pandemic influenza and viral hemorrhagic fever (filoviruses) like the Ebola virus.
